In [2]:
import tensorflow_hub as hub

In [3]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [6]:
from absl import logging
import numpy as np

#@title Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "elephant is animal"
paragraph = (
    "elephant is animal"
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

message_embeddings = embed(messages)

for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  print("Message: {}".format(messages[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Elephant
Embedding size: 512
Embedding: [0.008344483561813831, 0.0004808558733202517, 0.06595248728990555, ...]

Message: elephant is animal
Embedding size: 512
Embedding: [-0.02572762779891491, -0.00338289188221097, 0.06081477552652359, ...]

Message: elephant is animalThere is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [-0.02780488319694996, -0.028051231056451797, -0.013840027153491974, ...]



     |████████████████████████████████| 427 kB 6.9 MB/s eta 0:00:01


In [11]:
import numpy as np
from horapy import HNSWIndex

dimension = 512
n = 1000

# init index instance
index = HNSWIndex(dimension, "usize")

samples = np.float32(np.random.rand(n, dimension))
for i in range(0, len(samples)):
    # add node
    index.add(np.float32(samples[i]), i)

index.build("euclidean")  # build index

target = np.random.randint(0, n)
# 410 in Hora ANNIndex <HNSWIndexUsize> (dimension: 50, dtype: usize, max_item: 1000000, n_neigh: 32, n_neigh0: 64, ef_build: 20, ef_search: 500, has_deletion: False)
# has neighbors: [410, 736, 65, 36, 631, 83, 111, 254, 990, 161]
print("{} in {} \nhas neighbors: {}".format(
    target, index, index.search(samples[target], 10)))  # search


951 in Hora ANNIndex <HNSWIndexUsize> (dimension: 512, dtype: usize, max_item: 1000000, n_neigh: 32, n_neigh0: 64, ef_build: 20, ef_search: 500, has_deletion: False) 
has neighbors: [951, 576, 799, 987, 384, 495, 434, 348, 445, 632]


In [27]:
index = HNSWIndex(dimension, "usize")
knowledge_dict = dict()
NEWLINE = '\n'
def convert_one_file(filename, target_fd):
    """prefix list of keywords
    """
    buffer = list()
    progress = 0
    for line in open(filename):
        if line.strip() == '':
            if buffer:
                progress += 1
                message = ''.join(buffer)
                # print(buffer)
                message_embeddings = embed([message])
                message_embedding = np.array(message_embeddings).tolist()[0]
                # print(message_embedding)
                message_embedding_snippet = ", ".join((str(x) for x in message_embedding))
#                 v = f"embedding: {message_embedding_snippet}"
#                 target_fd.write(v + NEWLINE)
#                 target_fd.write(message)
#                 target_fd.write(NEWLINE)
                knowledge_dict[progress] = message
                index.add(np.float32(message_embedding), progress)
                buffer =  list()
                if progress % 10000 == 1:
                    print(progress)
        else:
            buffer.append(line)

convert_one_file('/home/a/knowledge/test.txt', open('testv.txt', 'w'))
print('build index')
index.build("euclidean")  # build index
print('done')

1
10001
20001
30001
40001
50001
build index
done


In [35]:
a = np.float32(np.random.rand(n, dimension))
query = 'deepest thoughts'
message_embeddings = embed([query])
message_embedding = np.array(message_embeddings).tolist()[0]

results = index.search(message_embedding, 10)
for i in results:
    print(knowledge_dict[i])

ideas learn-anything !
![](https://i.imgur.com/CXLG4IY.jpg)

computer-science our
[Our Thoughts on P=NP (2020)](https://rjlipton.wordpress.com/2020/01/12/our-thoughts-on-pnp/)

future ability
Ability to control computers with thoughts at high bandwidth.

ideas learn-anything build
[Build your own X](https://github.com/danistefanovic/build-your-own-x)

physics string-theory the biggest
[The Biggest Ideas in the Universe (2020)](https://www.youtube.com/playlist?list=PLrxfgDEc2NxZJcWcrxH3jyjUUrJlnoyzX)

future progress
[Progress](https://patrickcollison.com/progress)

ideas learn-anything are
[Are Ideas Getting Harder to Find? (2020)](https://web.stanford.edu/~chadj/IdeaPF.pdf)

math mathpages
[MathPages](https://www.mathpages.com/home/index.htm)

mindfulness meditation treating
Treating thoughts and thinking in this way is incredibly freeing. Not getting attached to what you think. And not getting attached to anything is incredibly empowering feeling. But it is something that you have to